In [1]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_pickle('./data/pickle/preproc/df_patient_admit_icu__20210204_singleICUSTAY_final.pkl')

In [3]:
# import patient info
data_dir = 'data/physionet.org/files/mimiciii/1.4/'
notes_file = 'NOTEEVENTS.csv'

In [29]:
skiplist = []

row_count = 700000
skiprows = 0
nrows = 100000  # defualt

colnames = np.array(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CHARTDATE',
           'CHARTTIME', 'STORETIME', 'CATEGORY', 'DESCRIPTION',
           'CGID', 'ISERROR','TEXT'
          ])

usecols = colnames[[1,2,4, 9, 10]]
cnt=0

In [30]:
def preprocess_text(df):
    # This function preprocesses the text by filling not a number and replacing new lines ('\n') and carriage returns ('\r')
    df.TEXT = df.TEXT.fillna(' ')
    df.TEXT = df.TEXT.str.replace('\n',' ')
    df.TEXT = df.TEXT.str.replace('\r',' ')
    return df

In [31]:
all_colnames = ['CHARTTIME', 'TEXT', 'SUBJECT_ID', 'GENDER', 'HADM_ID', 'ADMITTIME',
       'DISCHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'INSURANCE',
       'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY', 'DIAGNOSIS',
       'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA', 'HOSPITAL_DAYS',
       'ADMIT_AGE', 'ICUSTAY_ID', 'DBSOURCE', 'INTIME', 'LOS',
       'DAYS_ADM_TO_ICU', 'SAMEDAY_ADM_TO_ICU', 'ADM_TO_ICU_100p',
       'ADM_TO_ICU_90m', 'ICU_URGENCY', 'DAYS_NOTE_TO_ICU']

df_all = pd.DataFrame(columns = all_colnames)
while skiprows<row_count:
    print('Iteration {}...'.format(cnt))
    
    if skiprows + nrows > row_count:
        nrows = row_count - skiprows
    else:
        nrows = 100000
            
    df_note = pd.read_csv(data_dir + notes_file, sep=',', header=0, names=colnames,
                     skiprows=skiprows, nrows=nrows, usecols=usecols)
    
    # drop charttime=na
    df_note = df_note.dropna(subset=['CHARTTIME'])
    
    # drop note errors
    df_note = df_note[df_note['ISERROR']!=1] # 1 is error
    df_note = df_note.drop(columns=['ISERROR'])
    
    # keep only rows that have matching ['SUBJECT_ID', 'HADM_ID'] in df_ids
    df_note = df_note.merge(df, on=['SUBJECT_ID', 'HADM_ID'], how='inner')
    
    # convert charttime to datetime
    df_note.CHARTTIME = pd.to_datetime(df_note.CHARTTIME,format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
    
    # calculate days from note event to icu admission
    df_note['DAYS_NOTE_TO_ICU'] = (df_note['INTIME'] - df_note['CHARTTIME']).dt.total_seconds()/(24*60*60)
    
    # keep only chartevents that occurred before ICU INTIME (>0)
    df_note = df_note[df_note['DAYS_NOTE_TO_ICU']>0]
    
    # fix text
    df_note = preprocess_text(df_note)
    
    ## TO DO:
    # 1. for each SUBJECT_ID, find first entry time HADM_ID and create new row with empty TEXT 
    #    (indicating info received immediately after first admission)
    # 2. clean up TEXT (similar to DIAGNOSES) 
    # 3. for each SUBJECT_ID & HADM_ID, order rows by ADMITTIME, then CHARTTIME
    # 4. for every row after first for each  SUBJECT_ID (across all HADM_IDs): 
    #(
    #    just concatenate all subsequent note events for by SUBJECT_ID (this serves to extend 
    #    running tally of notes for each subject across hospital admits)
    
    #    or TEXT as running set() of words that gets extended at each later time for each new note 
    #    event, but converted back to text (with proper word counts) to use with CountVectorizer and TfidfTransformer 
    
    #)
    
    # 5. create new DAYS_EVENT_TO_ICU that collapses DAYS_ADM_TO_ICU (for admit info only) and DAYS_NOTE_TO_ICU (for all notes)
    df_all = df_all.append(df_note)
    
    skiprows+=nrows

Iteration 0...
Iteration 0...
Iteration 0...
Iteration 0...
Iteration 0...
Iteration 0...
Iteration 0...


In [46]:
df_all.head(5)

,CHARTTIME,TEXT,SUBJECT_ID,GENDER,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,INSURANCE,...,ICUSTAY_ID,DBSOURCE,INTIME,LOS,DAYS_ADM_TO_ICU,SAMEDAY_ADM_TO_ICU,ADM_TO_ICU_100p,ADM_TO_ICU_90m,ICU_URGENCY,DAYS_NOTE_TO_ICU
55422,2176-04-27 03:09:00,Pt is 72 yo M with PMHx sig. for cirrhosis s/p...,21666,M,174565.0,2176-04-26 12:00:00,2176-05-05 12:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,Medicare,...,283592,metavision,2176-04-29 22:05:46,0.5797,3.420671,0,86,54,questionable,2.789421
55423,2176-04-28 05:07:00,SICU HPI: Date [**4-28**] HD 2 POD 1 ...,21666,M,174565.0,2176-04-26 12:00:00,2176-05-05 12:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,Medicare,...,283592,metavision,2176-04-29 22:05:46,0.5797,3.420671,0,86,54,questionable,1.707477
55427,2176-04-28 09:53:00,"Gastrointestinal bleed, lower (Hematochezia, B...",21666,M,174565.0,2176-04-26 12:00:00,2176-05-05 12:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,Medicare,...,283592,metavision,2176-04-29 22:05:46,0.5797,3.420671,0,86,54,questionable,1.508866
55428,2176-04-27 18:01:00,Pt is 72 yo M with PMHx sig. for cirrhosis s/p...,21666,M,174565.0,2176-04-26 12:00:00,2176-05-05 12:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,Medicare,...,283592,metavision,2176-04-29 22:05:46,0.5797,3.420671,0,86,54,questionable,2.169977
55429,2176-04-28 09:53:00,"Gastrointestinal bleed, lower (Hematochezia, B...",21666,M,174565.0,2176-04-26 12:00:00,2176-05-05 12:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,Medicare,...,283592,metavision,2176-04-29 22:05:46,0.5797,3.420671,0,86,54,questionable,1.508866


In [74]:
preproc_dir = 'data/pickle/preproc/'
notes_rowids = preproc_dir + 'NOTEEVENTS__2021_01_22_22_49_08.pkl'
note_ids = pd.read_pickle(notes_rowids)

In [75]:
total_len = sum([len(i) for i in note_ids])

In [80]:
row_idx = set()
for n in note_ids:
    row_idx.update(n)

In [81]:
len(row_idx)

1776453

In [13]:
skiprows = 100000
foo = pd.read_csv(data_dir + notes_file, sep=',', header=0, names=colnames,
                     skiprows=skiprows, nrows=nrows, usecols=[1,2,4, 9])

In [15]:
mask = foo.ISERROR!=1

In [17]:
foo = foo[~mask]

In [18]:
foo.shape

(0, 4)

In [117]:
df.dropna(subset=['CHARTTIME'])

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTTIME


In [118]:
foo

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTTIME
0,99733,88065,160734.0,NaN
1,99734,88065,NaN,NaN
2,99829,4978,184712.0,NaN
3,99947,17294,196476.0,NaN
4,99948,19728,NaN,NaN
...,...,...,...,...
99995,209265,60054,154500.0,NaN
99996,209266,60054,154500.0,NaN
99997,209481,6639,186534.0,NaN
99998,209482,5688,144939.0,NaN


In [ ]:
# convert charttime to datetime
df.CHARTTIME = pd.to_datetime(df.CHARTTIME,format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')

In [101]:
2083181-200000-14931-465-487-506-163

1866629